In [1]:
import os
import os.path as path
import glob

from csv import (writer, DictWriter)

import pandas as pd
import math
import numpy as np

from drain3 import TemplateMiner
from drain3.template_miner_config import TemplateMinerConfig

from spellpy import spell
from fastai.text.all import Numericalize

import fasttext

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import (confusion_matrix, precision_score, recall_score, f1_score)

In [6]:
project_root = os.getcwd()

data_dir = 'data'
log_file_name = 'BGL.log'

config_dir = 'config'

input_dir = 'input'
spell_output_dir = 'output'

spell_input_dir = 'input/spell'
spell_output_dir = 'output/spell'

drain_input_dir = 'input/drain'
drain_output_dir = 'output/drain'

WINDOW_SIZE = 5

In [11]:
log_csv_file_name = 'BGL.csv'
log_csv_file = log_file = path.abspath(path.join(project_root, '../', input_dir, log_csv_file_name))

if os.path.exists(log_csv_file):
    os.remove(log_csv_file)

log_file = path.abspath(path.join(project_root, '../', data_dir, log_file_name))
logs = open(log_file, 'r')

with open(log_csv_file, 'a') as log_csv_file_obj:
    log_csv_writer_obj = writer(log_csv_file_obj)
    for line in logs:
        split_data = line.rstrip('\n').split(' ')
        if len(split_data) > 9:
            split_data[9] = ' '.join(split_data[9:])
            log_csv_writer_obj.writerow(split_data[0:10])
    log_csv_file_obj.close